In [16]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
import warnings
warnings.filterwarnings(action = 'ignore')



In [17]:
df = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 신호등 관련 정보.csv",encoding='CP949')

def get_traffic_light(lon, lat, km):
    df = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 신호등 관련 정보.csv",encoding='CP949')
    
    df['X좌표'] = df['X좌표'].astype(float)
    df['Y좌표'] = df['Y좌표'].astype(float)
    df['geometry'] = df.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry')
    df.crs = from_string("+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")
    wgs84 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df = df.to_crs(wgs84)
    df2 = pd.DataFrame()
    df2['lon'] = df['geometry'].map(lambda t : t.x)
    df2['lat'] = df['geometry'].map(lambda t : t.y)
    df2['신호등수량'] = df['신호등수량']
    df2 = df2.rename({'lat':'위도','lon':'경도'},axis='columns')
    df2 = df2[['신호등수량','위도','경도']]
    return get_xy(df2, lon, lat, km)

In [12]:
def get_traffic_light(lon, lat, km):
    global df,df2
    df = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 신호등 관련 정보.csv",encoding='CP949')
    df['X좌표'] = df['X좌표'].astype(float)
    df['Y좌표'] = df['Y좌표'].astype(float)
    df['geometry'] = df.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry')
    df.crs = from_string("+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")
    wgs84 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df = df.to_crs(wgs84)
    df['lon'] = df['geometry'].map(lambda t : t.x)
    df['lat'] = df['geometry'].map(lambda t : t.y)
#     df['신호등수량'] = df['신호등수량']
    df = df.rename({'lat':'위도','lon':'경도'},axis='columns')
    df = df[['신호등수량','위도','경도']]
    return get_xy(df, lon, lat, km)

target_tl = get_traffic_light(126.955009,37.6027953,2)
target_tl

/Users/chanju/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,신호등수량,위도,경도
386,4,37.599128,126.947608
387,4,37.599128,126.947608
388,4,37.599128,126.947608
700,1,37.610465,126.946898
702,1,37.597023,126.947607
...,...,...,...
108493,4,37.594476,126.946057
108589,2,37.595620,126.948895
109354,2,37.601823,126.961116
109355,2,37.602130,126.961135


In [376]:
lon =126.955009
lat =37.6027953
tmp = get_tlsghemd(lon,lat,2)
tmp

/Users/chanju/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,신호등수량,위도,경도
386,4,37.599128,126.947608
387,4,37.599128,126.947608
388,4,37.599128,126.947608
700,1,37.610465,126.946898
702,1,37.597023,126.947607
...,...,...,...
108493,4,37.594476,126.946057
108589,2,37.595620,126.948895
109354,2,37.601823,126.961116
109355,2,37.602130,126.961135


In [11]:
#위도 =111.195km , 경도 88.8km , 2km 박스일 때 반경  1.414
def dis_to_lat(km):
    return km/111.195
def dis_to_lon(km):
    return km/88.8

def lat_to_dis(lat):
    return lat*111.195
def lon_to_dis(lon):
    return lon*88.8

def get_xy(data,lon, lat,km):
    lat_size = dis_to_lat(km)
    lon_size = dis_to_lon(km)
    return data[(data['위도']<=lat+lat_size/2) & (data['위도']>=lat-lat_size/2) &
                 (data['경도']<=lon+lon_size/2) & (data['경도']>=lon-lon_size/2)]

In [130]:
df.head(5)

,부착대관리번호,상태 (공통),부착방식,부착대길이,고가 (공통),부착대방향,신호등수량,배면등수량,신호등종류,배면등종류,...,작업구분 (공통),표출구분 (공통),신규정규화ID,공사관리번호,부착대관리번호.1,이력ID,위치정보,X좌표,Y좌표,공사형태 (공통)
0,03-0000034286,001,003,009,001,195,2,0,003,002,...,4,001,,2004-1101-152,03-034286,88931,,202650.00560,542705.46253,
1,03-0000044336,001,003,011,001,102,3,0,003,002,...,2,001,,2004-1101-152,03-044336,88932,,202714.48048,542715.77569,
2,03-0000044307,001,002,999,001,312,1,0,007,,...,1,002,,2008-0201-006,03-044307,88933,,203129.34784,541838.75856,
3,03-0000048361,001,002,999,001,288,1,0,007,,...,1,002,,2008-0201-006,03-048361,88934,,202976.86616,542798.94598,
4,03-0000015050,001,002,999,001,302,1,0,007,,...,1,002,,2008-0201-006,03-015050,88935,,203026.11371,542800.32686,


In [131]:
df.columns

Index(['부착대관리번호', '상태 (공통)', '부착방식', '부착대길이', '고가 (공통)', '부착대방향', '신호등수량',
       '배면등수량', '신호등종류', '배면등종류', '설치일', '교체일', '지주관리번호', '신호발광구분', '제조회사',
       '작업구분 (공통)', '표출구분 (공통)', '신규정규화ID', '공사관리번호', '부착대관리번호.1', '이력ID',
       '위치정보', 'X좌표', 'Y좌표', '공사형태 (공통)'],
      dtype='object')

In [132]:
df = df.drop(['상태 (공통)', '부착방식', '부착대길이', '고가 (공통)', '부착대방향', '배면등수량',
       '신호등종류', '배면등종류', '설치일', '교체일', '지주관리번호', '신호발광구분', '제조회사', '작업구분 (공통)',
       '표출구분 (공통)', '신규정규화ID', '공사관리번호', '부착대관리번호', '이력ID', '위치정보',
       '공사형태 (공통)','부착대관리번호.1'],axis='columns')

In [133]:
df.head(5)

,신호등수량,X좌표,Y좌표
0,2,202650.00560,542705.46253
1,3,202714.48048,542715.77569
2,1,203129.34784,541838.75856
3,1,202976.86616,542798.94598
4,1,203026.11371,542800.32686


In [135]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString


df['X좌표'] = df['X좌표'].astype(float)
df['Y좌표'] = df['Y좌표'].astype(float)
df['geometry'] = df.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
df = gpd.GeoDataFrame(df, geometry='geometry')

In [136]:
from fiona.crs import from_string

epsg5186 = from_string("+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")

In [137]:
df.crs = epsg5186
print(df.crs)

+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs +type=crs


In [138]:
wgs84 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [139]:
df = df.to_crs(wgs84)
df["geometry"].head()

0    POINT (127.02996 37.48379)
1    POINT (127.03069 37.48388)
2    POINT (127.03538 37.47598)
3    POINT (127.03366 37.48463)
4    POINT (127.03422 37.48464)
Name: geometry, dtype: geometry

In [140]:
df.head()

,신호등수량,X좌표,Y좌표,geometry
0,2,202650.00560,542705.46253,POINT (127.02996 37.48379)
1,3,202714.48048,542715.77569,POINT (127.03069 37.48388)
2,1,203129.34784,541838.75856,POINT (127.03538 37.47598)
3,1,202976.86616,542798.94598,POINT (127.03366 37.48463)
4,1,203026.11371,542800.32686,POINT (127.03422 37.48464)


In [161]:
df.iloc[1]['geometry'].x
df2 = pd.DataFrame()
df2['lon'] = df['geometry'].map(lambda t : t.x)
df2['lat'] = df['geometry'].map(lambda t : t.y)
df2['신호등수량'] = df['신호등수량']

In [162]:
df2

,lon,lat,신호등수량
0,127.029962,37.483790,2
1,127.030691,37.483883,3
2,127.035378,37.475980,1
3,127.033658,37.484631,1
4,127.034215,37.484644,1
...,...,...,...
109995,126.818338,37.487184,1
109996,127.070194,37.631123,1
109997,127.057387,37.670952,1
109998,127.058284,37.671118,1


In [163]:
df2 = df2.rename({'lat':'위도','lon':'경도'},axis='columns')
df2 = df2[['신호등수량','위도','경도']]

In [170]:
lon =126.955009
lat =37.6027953

tmp1 = get_xy(df2, lon, lat, 2) # return dataframe 
tmp2 = get_xy(df2, lon, lat, 1)

print(tmp1,tmp1['신호등수량'].sum())
print(tmp2,tmp2['신호등수량'].sum())

        신호등수량         위도          경도
386         4  37.599128  126.947608
387         4  37.599128  126.947608
388         4  37.599128  126.947608
700         1  37.610465  126.946898
702         1  37.597023  126.947607
...       ...        ...         ...
108493      4  37.594476  126.946057
108589      2  37.595620  126.948895
109354      2  37.601823  126.961116
109355      2  37.602130  126.961135
109831      1  37.595615  126.948908

[478 rows x 3 columns] 704
        신호등수량         위도          경도
3019        2  37.606869  126.958980
4769        2  37.600049  126.958185
5196        4  37.600815  126.956923
5197        2  37.600815  126.956923
5198        1  37.599976  126.958122
...       ...        ...         ...
106828      1  37.599943  126.957731
107178      1  37.600049  126.958185
107181      2  37.600074  126.957837
107214      1  37.599875  126.957874
107292      1  37.600049  126.958185

[73 rows x 3 columns] 115


In [2]:
def get_crosswalk(lon,lat,km) :
    df3 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 횡단보도 위치정보 (좌표계_ WGS1984).csv",encoding='CP949')
    df3 = df3.drop(['순번', '상태', '횡단보도종류코드', '가로길이', '세로길이', '화살표시수량', '화살표시길이',
       '고가', '구경찰서코드', '구코드', '동코드', '지번', '신경찰서코드', '작업구분', '표출구분', '도로구분',
       '관할사업소', '신규정규화ID', '설치일', '교체일', '이력ID', '공사관리번호', '횡단보도관리번호.1',
       '공사형태'], axis='columns')
    return get_xy(df3, lon, lat, km)

In [312]:
df3 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 횡단보도 위치정보 (좌표계_ WGS1984).csv",encoding='CP949')

In [313]:
df3.head()

,순번,횡단보도관리번호,상태,횡단보도종류코드,가로길이,세로길이,화살표시수량,화살표시길이,고가,구경찰서코드,...,관할사업소,신규정규화ID,설치일,교체일,이력ID,공사관리번호,횡단보도관리번호.1,공사형태,위도,경도
0,76,06-0000030057,001,002,0.0,0.0,4,001,001,390,...,108,'','','',47540.0,2000-0000-000,06-030057,'',37.631108,126.927777
1,77,06-0000029813,001,001,0.0,0.0,0,001,001,240,...,108,'','','',39989.0,2008-0104-956,06-029813,005,37.578073,126.927166
2,78,06-0000029814,001,001,0.0,0.0,0,001,001,240,...,108,'','','',39990.0,2008-0104-956,06-029814,005,37.577637,126.927554
3,79,06-0000029815,001,001,0.0,0.0,0,001,001,240,...,108,'','','',39991.0,2008-0104-956,06-029815,005,37.577401,126.927198
4,81,06-0000029818,001,001,0.0,0.0,0,001,001,240,...,108,'','','',39994.0,2008-0104-956,06-029818,005,37.577929,126.928199


In [314]:
df3.columns

Index(['순번', '횡단보도관리번호', '상태', '횡단보도종류코드', '가로길이', '세로길이', '화살표시수량', '화살표시길이',
       '고가', '구경찰서코드', '구코드', '동코드', '지번', '신경찰서코드', '작업구분', '표출구분', '도로구분',
       '관할사업소', '신규정규화ID', '설치일', '교체일', '이력ID', '공사관리번호', '횡단보도관리번호.1',
       '공사형태', '위도', '경도'],
      dtype='object')

In [332]:
df3 = df3.drop(['순번', '상태', '횡단보도종류코드', '가로길이', '세로길이', '화살표시수량', '화살표시길이',
       '고가', '구경찰서코드', '구코드', '동코드', '지번', '신경찰서코드', '작업구분', '표출구분', '도로구분',
       '관할사업소', '신규정규화ID', '설치일', '교체일', '이력ID', '공사관리번호', '횡단보도관리번호.1',
       '공사형태'], axis='columns')

In [2]:
tmp3 = get_xy(df3, lon, lat, 2) # return dataframe 
tmp4 = get_xy(df3, lon, lat, 1)
print(tmp3,sep='\n')
print(tmp4,len(tmp4))

NameError: name 'get_xy' is not defined

In [1]:
def get_bus_stop(lon,lat,km) :
    df4 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울특별시 버스정류소 위치정보.csv",encoding='CP949')
    df4 = df4.drop(['정류소번호'],axis=1)
    df4 = df4.rename({'X좌표':'경도','Y좌표':'위도'},axis=1)
    df4 = df4[['정류소명','위도','경도']]
    
    return get_xy(df4,lon,lat,km)

In [334]:
df4 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울특별시 버스정류소 위치정보.csv",encoding='CP949')


In [335]:
df4.head()

,정류소번호,정류소명,X좌표,Y좌표
0,1001,종로2가사거리,126.987786,37.569764
1,1002,창경궁.서울대학교병원,126.996520,37.579179
2,1003,명륜3가.성대입구,126.998290,37.582709
3,1004,종로2가.삼일교,126.987507,37.568582
4,1005,혜화동로터리.여운형활동터,127.001694,37.586230


In [339]:
df4 = df4.drop(['정류소번호'],axis=1)

In [343]:
df4 = df4.rename({'X좌표':'경도','Y좌표':'위도'},axis=1)
df4 = df4[['정류소명','위도','경도']]

In [368]:
tmp5 = get_xy(df4, lon, lat, 2) # return dataframe 
tmp6 = get_xy(df4, lon, lat, 1)
print(tmp5,sep='\n')
print(tmp6,sep='\n')
print("범위 내 정류장의 갯수는 %d개 입니다."%len(tmp6))

             정류소명         위도          경도
54      국민은행세검정지점  37.604688  126.960969
55        세검정초등학교  37.602488  126.961284
56        세검정.상명대  37.600665  126.958771
57      상명대입구.석파랑  37.599605  126.958688
58            하림각  37.598269  126.961828
...           ...        ...         ...
5160       홍지문35통  37.596732  126.956360
5161         옥천교앞  37.596341  126.954236
5168  동원베네스트아파트입구  37.598247  126.944763
5169    홍은유원아파트입구  37.597321  126.945430
5170    홍은사회복지관별관  37.593959  126.947757

[99 rows x 3 columns]
             정류소명         위도          경도
56        세검정.상명대  37.600665  126.958771
57      상명대입구.석파랑  37.599605  126.958688
64    상명대입구.세검정교회  37.599560  126.959219
65        세검정.상명대  37.600615  126.958889
72            구기동  37.607045  126.958597
73            구기동  37.606954  126.959056
144         상명대정문  37.601666  126.955066
146   상명대입구.세검정교회  37.599393  126.959766
147     상명대입구.석파랑  37.599767  126.958352
218           문방구  37.606297  126.957949
224          산정빌라  37.605153  126.

In [3]:
def get_subway(lon,lat,km) :
    df5 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/station_coordinate.csv")
    df5 = df5.drop(['line','code'],axis=1)
    df5 = df5.rename({'lat':'위도','lng':'경도'},axis=1)
    return get_xy(df5,lon,lat,km)

In [396]:
df5 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/station_coordinate.csv")

In [397]:
df5.head()

,line,name,code,lat,lng
0,01호선,녹양,1908.0,37.759380,127.042292
1,01호선,남영,1002.0,37.541021,126.971300
2,01호선,용산,1003.0,37.529849,126.964561
3,01호선,노량진,1004.0,37.514219,126.942454
4,01호선,대방,1005.0,37.513342,126.926382


In [398]:
df5.columns

Index(['line', 'name', 'code', 'lat', 'lng'], dtype='object')

In [399]:
df5 = df5.drop(['line','code'],axis=1)
df5 = df5.rename({'lat':'위도','lng':'경도'},axis=1)
df5.head()

,name,위도,경도
0,녹양,37.759380,127.042292
1,남영,37.541021,126.971300
2,용산,37.529849,126.964561
3,노량진,37.514219,126.942454
4,대방,37.513342,126.926382


In [405]:
tmp7 = get_xy(df5, lon, lat, 6) # return dataframe 
tmp7

,name,위도,경도
167,안국,37.576477,126.985443
173,불광,37.610469,126.929887
174,녹번,37.600927,126.935756
175,홍제,37.589066,126.943736
176,무악재,37.582299,126.950291
304,역촌,37.606021,126.922744
305,불광,37.610469,126.929887
306,독바위,37.618456,126.933031
